## IMPORT USEFUL LIBRARIES AND PACKAGES

In [2]:
import glob
import os
import sys
import pandas as pd
import numpy as np
import json

partial_path = os.getcwd().rsplit('/', 1)[0]
sys.path.insert(0, f'{partial_path}/src/')

from utils import crop_damage_v1
import config as cf

## LOAD JSON FILE CONTAINING ANNOTATIONS

In [3]:
with open(cf.DAMAGE_DATASET_ANNOTPATH) as f:
    damage_bdeo = json.load(f)

## POST-PROCESS ANNOTATION FILE 

In [4]:
df_annotations = pd.DataFrame.from_dict(damage_bdeo['annotations'])
df_annotations.reset_index(inplace=True, drop=True)
im_dict = {}

for im in damage_bdeo['images']:
    im_dict[im['id']] = im['file_name']

df_annotations['path'] = df_annotations['image_id'].apply(lambda x: im_dict[x])
df_annot = df_annotations.filter(['id', 'bbox', 'path'])
df_annot['bbox_list'] = df_annot['bbox'].apply(lambda x: [x])
df_annot_grouped_by_image = df_annot.groupby(['path'], as_index=False).agg({'bbox_list':np.sum})
dict_annot_grouped_by_image = dict(zip(df_annot_grouped_by_image['path'], df_annot_grouped_by_image['bbox_list']))

## APPLY BBOX FROM ANNOTATIONS TO CROP DAMAGES OUT OF IMAGES

In [18]:
for index, image in enumerate(glob.glob(cf.DAMAGE_DATASET_IMAGEPATH)):
    crop_damage_v1(image, df_annot['bbox_list'][index], title=os.path.basename(image), save=True, outpath=cf.DAMAGE_CROP_IMAGEPATH)